# Import Dependencies and data 
#### Read the data using pandas 

In [29]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np
from sklearn import tree

# Study data files
data1 = "../Data/nhl odds 2014-15.csv"
data2 = "../Data/nhl odds 2015-16.csv"
data3 = "../Data/nhl odds 2016-17.csv"
data4 = "../Data/nhl odds 2017-18.csv"
data5 = "../Data/nhl odds 2018-19.csv"
data6 = "../Data/nhl odds 2019-20.csv"

# Read the mouse data and the study results
data1 = pd.read_csv(data1)
data2 = pd.read_csv(data2)
data3 = pd.read_csv(data3)
data4 = pd.read_csv(data4)
data5 = pd.read_csv(data5)
data6 = pd.read_csv(data6)

data6 = data6.rename(columns={"PuckLine": "Puck Line", "OpenOU": "Open OU", "CloseOU": "Close OU"})

frames = [data1, data2, data3, data4, data5, data6]


# Merge all the different seasons 

In [30]:
def set_win(row):
    if row['Point Dif'] > 0:
        return 1
    else:
        return 0

# Create row to see who won or loss by checking the point diferential was positive or negative.
def set_win_pl(row):
    if row['Point Dif'] >= 2:
        return 1
    elif (row['Point Dif'] >= -1) & (row['Puck Line'] == 1.5):
        return 1
    else:
        return 0

def data_clean(df):
    df = df.replace({'Team': {'Arizonas': 'Arizona', 'NewJersey': 'New Jersey', 'NYIslanders': 'NY Islanders', 'NYRangers': 'NY Rangers',  'SanJose': 'San Jose',  'St.Louis': 'St. Louis', 'TampaBay': 'Tampa Bay','LosAngeles':'Los Angeles'}})
    p_dif = []

    # Iterate through table and calculate point dif and insert into list 
    for i in range(1, df.shape[0], 2):
        home_dif = df['Final'].iloc[i] - df['Final'].iloc[i-1]
        away_dif = df['Final'].iloc[i-1] - df['Final'].iloc[i]
        p_dif.append(away_dif)
        p_dif.append(home_dif)

    # Add point dif column and data         
    df['Point Dif'] = p_dif
    df = df.reset_index()
    # Create point differential column 
    line_move = []

    # Iterate through table and calculate point dif and insert into list 
    for i in range(len(df)):
        move = df['Open'].iloc[i] - df['Close'].iloc[i]
        line_move.append(move)

    # Add point dif column and data         
    df['Line Mvmt'] = line_move
    df = df.reset_index()
    
    # add the result column 
    df = df.assign(Win=df.apply(set_win, axis=1))
    df = df.assign(Win_PL=df.apply(set_win_pl, axis=1)) 
    
    goal_dict = {}
    net_goals = []

    for i in df.iterrows():
    #point difference
        if i[1]['Team'] in goal_dict:
            net_goals.append(goal_dict[i[1]['Team']])
            goal_dict[i[1]['Team']] = goal_dict[i[1]['Team']] + int(i[1]['Point Dif'])
        else:
            goal_dict[i[1]['Team']] = int(i[1]['Point Dif'])
            net_goals.append(0)
            
    df['net_goals'] = net_goals
    return df

In [31]:
new_frames = []
for frame in frames:
    clean = data_clean(frame)
    new_frames.append(clean)
    
df = pd.concat(new_frames)
cols = [0,1]
df.drop(df.columns[cols],axis=1,inplace=True)
df
df

,Date,Rot,VH,Team,1st,2nd,3rd,Final,Open,Close,...,Unnamed: 11,Open OU,Unnamed: 13,Close OU,Unnamed: 15,Point Dif,Line Mvmt,Win,Win_PL,net_goals
0,1008,1,V,Montreal,1,1,2,4,-105,-105,...,-305.0,5.5,106,5.5,110,1,0,1,1,0
1,1008,2,H,Toronto,2,0,1,3,-115,-115,...,250.0,5.5,-126,5.5,-130,-1,0,0,0,0
2,1008,3,V,Philadelphia,0,0,1,1,165,165,...,-180.0,5.5,123,5.5,115,-1,0,0,1,0
3,1008,4,H,Boston,1,0,1,2,-195,-185,...,155.0,5.5,-143,5.5,-135,1,-10,1,0,0
4,1008,5,V,San Jose,1,3,0,4,101,130,...,-240.0,5.5,127,5.0,-136,4,-29,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2159,311,66,H,Colorado,1,1,0,3,-165,-146,...,170.0,6.0,-110,6.0,-115,1,-19,1,0,45
2160,311,67,V,Ottawa,1,0,1,2,140,145,...,-180.0,5.5,-110,6.0,-105,-1,-5,0,1,-51
2161,311,68,H,Los Angeles,1,0,2,3,-160,-160,...,160.0,5.5,-110,6.0,-115,1,0,1,0,-35
2162,311,89,V,St. Louis,2,1,1,4,-177,-200,...,115.0,6.0,-110,6.5,-110,2,23,1,1,30


In [32]:
df.to_csv('data.csv')

# Cleaning Up the Dataframe

In [9]:
cols = [0,1,3,4,5,6,7,8,12,13,14,15,16]
df.drop(df.columns[cols],axis=1,inplace=True)
df

,VH,Close,Puck Line,Unnamed: 11,Line Mvmt,Win,Win_PL,net_goals
0,V,-105,1.5,-305.0,0,1,1,0
1,H,-115,-1.5,250.0,0,0,0,0
2,V,165,1.5,-180.0,0,0,1,0
3,H,-185,-1.5,155.0,-10,1,0,0
4,V,130,1.5,-240.0,-29,1,1,0
...,...,...,...,...,...,...,...,...
2159,H,-146,-1.5,170.0,-19,1,0,45
2160,V,145,1.5,-180.0,-5,0,1,-51
2161,H,-160,-1.5,160.0,0,1,0,-35
2162,V,-200,-1.5,115.0,23,1,1,30


In [10]:
df = df.replace({'VH': {'V': 0, 'H': 1}})
df = df.rename(columns={"VH": "Home","Close": "ML", "Unnamed: 11": "PL"})
df = df[["Home", "ML", "Puck Line", "PL", "Line Mvmt", "net_goals", "Win", "Win_PL"]]
df = df.dropna()

df

,Home,ML,Puck Line,PL,Line Mvmt,net_goals,Win,Win_PL
0,0,-105,1.5,-305.0,0,0,1,1
1,1,-115,-1.5,250.0,0,0,0,0
2,0,165,1.5,-180.0,0,0,0,1
3,1,-185,-1.5,155.0,-10,0,1,0
4,0,130,1.5,-240.0,-29,0,1,1
...,...,...,...,...,...,...,...,...
2159,1,-146,-1.5,170.0,-19,45,1,0
2160,0,145,1.5,-180.0,-5,-51,0,1
2161,1,-160,-1.5,160.0,0,-35,1,0
2162,0,-200,-1.5,115.0,23,30,1,1


In [11]:
df.to_csv('final_df.csv')

# Random Forest Model

In [ ]:
#target = df["Win_PL"]
target = df["Win"]
target_names = ["loss", "win"]

In [ ]:
data = df.drop(columns=['Win', 'Win_PL'],axis=1)
feature_names = data.columns

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [150,160,170,175,180,190],
    'criterion': ['gini','entropy']
}

grid = GridSearchCV(RandomForestClassifier(), param_grid, verbose=True, n_jobs=-1)

grid.fit(X_train, y_train)

In [ ]:
print(grid.best_params_)

In [ ]:
rf = RandomForestClassifier(n_estimators=180)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

In [ ]:
# sorted(zip(rf.feature_importances_, feature_names), reverse=True)

# Deep Network

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=18, activation='relu', input_dim=6))
deep_model.add(Dense(units=9, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))

In [ ]:
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [ ]:
deep_model.fit(
    X_train,
    y_train_categorical,
    epochs=50,
    shuffle=True,
    verbose=2
)

In [ ]:
model_loss, model_accuracy = deep_model.evaluate(
    X_test, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [22]:
import pandas as pd
import re

In [23]:
url = 'https://www.pro-football-reference.com/years/2007/games.htm'

In [24]:
tables = pd.read_html(url)
tables

ImportError: lxml not found, please install it